# Purpose
研究婚外情的數據

(1) 該位外遇的可能性有多大？ 
(2) 影響外遇的原因是什麼？ 
(3) 可以根據分析結果說明如何降低外遇的發生嗎？

# 數據說明

* 婚外情數據即著名的“Fair’s Affairs”，取自於1969年《今日心理》（Psychology Today）所做的一個代表性調查
* 該數據從601個參與者身上收集了9個變量
* 變數包括一年來婚外情的頻率以及參與者性別、年齡、婚齡、是否有小孩、宗教信仰程度、學歷、職業，還有對婚姻的自我評分

#### affairs 受訪者在過去一年中進行外遇的頻率         
0 = 無         
1 = 一次       
2 = 兩次         
3 = 三次        
7 = 4 - 10 次         
12 = 每月或更多         

#### gender          
0 = 女性         
1 = 男性         

#### age         
17.5 = 20 歲以下         
22.0 = 20 - 24         
27.0 = 25 - 29         
32.0 = 30 - 34         
37.0 = 35 - 39         
42.0 = 40 - 44         
47.0 = 45 - 49         
52.0 = 50 - 54         
57.0 = 55 或以上         

#### yearsmarried  婚姻時間                  
0.125 = 3 個月或更短         
0.417 = 4 - 6 個月         
0.750 = 6 個月 - 1 年         
1.500 = 1 - 2 年         
4.000 = 3 - 5 年         
7.000 = 6 - 8 年         
10.00 = 9 - 11 年         
15.00 = 12 年或更長時間         

#### children 孩子人數                  
0 = 無         
1 = 一個或多個         
         
#### religiousness 婚內的宗教信仰         
1 = 反宗教         
2 = 完全沒有         
3 = 輕微         
4 = 有點         
5 = 非常         

#### education 教育程度         
9.0 = 小學         
12.0 = 高中畢業         
14.0 = 一些大學         
16.0 = 大學畢業生         
17.0 = 一些畢業作品         
18.0 = 碩士學位         
20.0 = 博士、醫學博士或其他高級學位 

#### occupation 詳見data-descriptions.pdf

#### rating 對婚姻的自我評分 (5分制，1表示非常不幸福，5表示非常幸福）         

# 1. import data

In [1]:
#Standard libraries for data analysis:  
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import norm, skew
from scipy import stats
import statsmodels.api as sm

import seaborn as sns # For creating plots
sns.set_style('darkgrid')

# sklearn modules for data preprocessing:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#sklearn modules for Model Selection:
from sklearn import svm, tree, linear_model, neighbors
from sklearn import naive_bayes, ensemble, discriminant_analysis, gaussian_process
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

#sklearn modules for Model Evaluation & Improvement:   
from sklearn.metrics import confusion_matrix, accuracy_score 
from sklearn.metrics import f1_score, precision_score, recall_score, fbeta_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import classification_report, precision_recall_curve
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.metrics import make_scorer, recall_score, log_loss
from sklearn.metrics import average_precision_score

#Standard libraries for data visualization:
import seaborn as sn
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import matplotlib 
%matplotlib inline
color = sn.color_palette()

import matplotlib.ticker as mtick
from IPython.display import display
pd.options.display.max_columns = None
from pandas.plotting import scatter_matrix
from sklearn.metrics import roc_curve

#Miscellaneous Utilitiy Libraries:
import random
import os
import re
import sys
import timeit
import string
import time
from datetime import datetime
from time import time
from dateutil.parser import parse
import joblib

In [2]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd

In [3]:
df = pd.read_csv('Affairs.csv')
df.head()

,affairs,gender,age,yearsmarried,children,religiousness,education,occupation,rating
0,0,male,37.0,10.00,no,3,18,7,4
1,0,female,27.0,4.00,no,4,14,6,4
2,0,female,32.0,15.00,yes,1,12,1,4
3,0,male,57.0,15.00,yes,5,18,6,5
4,0,male,22.0,0.75,no,2,17,6,3


### <span style="color:#3498DB">Point: 請先將affairs變數轉為二值型因子'ynaffair'，外遇0次為0，外遇一次以上為1</span>

In [4]:
df.info

<bound method DataFrame.info of      affairs  gender   age  yearsmarried children  religiousness  education  \
0          0    male  37.0         10.00       no              3         18   
1          0  female  27.0          4.00       no              4         14   
2          0  female  32.0         15.00      yes              1         12   
3          0    male  57.0         15.00      yes              5         18   
4          0    male  22.0          0.75       no              2         17   
..       ...     ...   ...           ...      ...            ...        ...   
596        1    male  22.0          1.50      yes              1         12   
597        7  female  32.0         10.00      yes              2         18   
598        2    male  32.0         10.00      yes              2         17   
599        2    male  22.0          7.00      yes              3         18   
600        1  female  32.0         15.00      yes              3         14   

     occupation  ra

In [5]:
#使用 Numpy 的 where (更通用，適合複雜條件)
# 語法：np.where(條件, 符合時的值, 不符合時的值)
df['y'] = np.where(df['affairs'] > 0, 1, 0)
print(df[['affairs', 'y']].head())
print(df['y'].value_counts()) # 觀察 0 和 1 的比例，確認是否有資料不平衡 (Imbalance)
df.drop(columns=['affairs'], inplace=True) 
# 步驟 2: 把 'y' 改名為 'affairs'
df.rename(columns={'y': 'affairs'}, inplace=True)
df

   affairs  y
0        0  0
1        0  0
2        0  0
3        0  0
4        0  0
y
0    451
1    150
Name: count, dtype: int64


,gender,age,yearsmarried,children,religiousness,education,occupation,rating,affairs
0,male,37.0,10.00,no,3,18,7,4,0
1,female,27.0,4.00,no,4,14,6,4,0
2,female,32.0,15.00,yes,1,12,1,4,0
3,male,57.0,15.00,yes,5,18,6,5,0
4,male,22.0,0.75,no,2,17,6,3,0
...,...,...,...,...,...,...,...,...,...
596,male,22.0,1.50,yes,1,12,2,5,1
597,female,32.0,10.00,yes,2,18,5,4,1
598,male,32.0,10.00,yes,2,17,6,5,1
599,male,22.0,7.00,yes,3,18,6,2,1


In [6]:
df_dummies = pd.get_dummies(df,dtype=int)
df_dummies.head()

,age,yearsmarried,religiousness,education,occupation,rating,affairs,gender_female,gender_male,children_no,children_yes
0,37.0,10.00,3,18,7,4,0,0,1,1,0
1,27.0,4.00,4,14,6,4,0,1,0,1,0
2,32.0,15.00,1,12,1,4,0,1,0,0,1
3,57.0,15.00,5,18,6,5,0,0,1,0,1
4,22.0,0.75,2,17,6,3,0,0,1,1,0


In [7]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split

# # 1. 分離 X 和 y (這就是你問的：先拉出來)
# y = df_dummies['affairs']                  # 目標變數 (不動它)
# X = df_dummies.drop('affairs', axis=1)     # 特徵變數 (準備被標準化)

# # 2. 切分訓練集與測試集 (建議在這步做，這是 Best Practice)
# # 假設我們切 80% 訓練，20% 測試
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # 3. 建立標準化器
# scaler = StandardScaler()

# # 4. 只用訓練資料來學習 "標準" (Fit)，並轉換 (Transform)
# X_train_scaled = scaler.fit_transform(X_train)

# # 5. 用同樣的標準去轉換測試資料 (注意：這裡只用 transform，不能 fit！)
# X_test_scaled = scaler.transform(X_test)

# # 此時，X_train_scaled 是 numpy array，若需要看資料可轉回 DataFrame
# # 但通常直接丟進模型訓練即可，不需要 append 回 y


from sklearn.preprocessing import MinMaxScaler  # <--- 改引用這個
from sklearn.model_selection import train_test_split

# 1. 分離 X 和 y (維持不變)
y = df_dummies['affairs']
X = df_dummies.drop('affairs', axis=1)

# 2. 切分訓練集與測試集 (維持不變)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. 建立 "MinMax" 縮放器 (這裡改了)
# feature_range=(0, 1) 是預設值，你也可以改成 (-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1)) 

# 4. 只用訓練資料來學習 "極值" (Fit)，並轉換 (Transform)
X_train_scaled = scaler.fit_transform(X_train)

# 5. 用同樣的極值去轉換測試資料
X_test_scaled = scaler.transform(X_test)

# 檢查結果：現在所有資料應該都在 0 到 1 之間了
print("最大值:", X_train_scaled.max())
print("最小值:", X_train_scaled.min())




最大值: 1.0
最小值: 0.0


In [ ]:
# 假設你的 X_train_scaled 已經算好了
# 1. 把它變回 DataFrame，並把原本的欄位名稱 (X.columns) 貼回去
X_train_view = pd.DataFrame(X_train_scaled, columns=X.columns)

# 2. 查看前 5 筆資料 (觀察數值是否都在 0~1 之間)
print("=== 資料預覽 ===")
print(X_train_view.head())

# 3. 查看統計摘要 (這是檢查標準化最重要的一步！)
# 重點看 min 是否為 0，max 是否為 1
print("\n=== 檢查是否成功壓縮到 0-1 ===")
print(X_train_view.describe().round(2))

=== 資料預覽 ===
        age  yearsmarried  religiousness  education  occupation  rating  \
0  0.113924      0.092437           0.75   0.454545    0.500000    1.00   
1  0.113924      0.092437           0.75   0.454545    0.666667    0.75   
2  0.113924      0.260504           0.75   0.454545    0.166667    1.00   
3  0.367089      0.663866           0.75   0.636364    0.500000    1.00   
4  0.620253      1.000000           0.75   0.818182    0.833333    1.00   

   gender_female  gender_male  children_no  children_yes  
0            0.0          1.0          1.0           0.0  
1            1.0          0.0          0.0           1.0  
2            0.0          1.0          1.0           0.0  
3            0.0          1.0          0.0           1.0  
4            0.0          1.0          0.0           1.0  

=== 檢查是否成功壓縮到 0-1 ===
          age  yearsmarried  religiousness  education  occupation  rating  \
count  480.00        480.00         480.00     480.00      480.00  480.00   
mean 

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("訓練集筆數:", len(X_train))
print("測試集筆數:", len(X_test))

訓練集筆數: 420
測試集筆數: 181


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

# --- 1. 訓練模型 ---
# 建立並訓練 Logistic Regression 模型
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)

# 進行預測
y_pred = lr.predict(X_test)

# --- 2. 畫出混淆矩陣 (Confusion Matrix) ---
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(6, 5))
# 畫熱力圖: annot=True顯示數字, fmt='d'整數格式, cmap='Blues'藍色系
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Affair (0)', 'Affair (1)'],
            yticklabels=['No Affair (0)', 'Affair (1)'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Logistic Regression')
plt.show()

# --- 3. 計算 4 大指標 ---
tn, fp, fn, tp = cm.ravel()

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0) # Sensitivity
specificity = tn / (tn + fp)

print("=== 模型評估指標 ===")
print(f"Accuracy (準確率): {accuracy:.3f}")
print(f"Precision (精確率): {precision:.3f}")
print(f"Recall (靈敏度/抓到外遇): {recall:.3f}")
print(f"Specificity (特異度/沒誤判): {specificity:.3f}")

# --- 4. 影響力分析 (特徵係數圖) ---
# 抓出係數並對應特徵名稱
coefs = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': lr.coef_[0]
})

# 排序以便觀察 (由小到大)
coefs = coefs.sort_values(by='Coefficient')

# 畫圖
plt.figure(figsize=(10, 6))
# 根據係數正負給顏色 (紅色=正向/增加外遇機率, 藍色=負向/降低外遇機率)
colors = ['blue' if x < 0 else 'red' for x in coefs['Coefficient']]
plt.barh(coefs['Feature'], coefs['Coefficient'], color=colors)
plt.axvline(0, color='black', linewidth=0.8) # 畫中間基準線
plt.title('Feature Importance (Logistic Regression Coefficients)')
plt.xlabel('Coefficient Value (Left: Decrease Risk, Right: Increase Risk)')
plt.show()

# 印出具體數值供參考
print("\n=== 特徵係數數值 (影響力排行) ===")
print(coefs.sort_values(by='Coefficient', ascending=False))

                 Model  Accuracy  Precision  Recall  Specificity
0  Logistic Regression     0.757      0.444   0.190        0.928
1                  SVM     0.768      0.000   0.000        1.000
2        Decision Tree     0.663      0.306   0.357        0.755
